추가적인 코디정보를 크롤링하기 위해서 만들어진 주피터 노트북입니다.

In [ ]:
import os
import pickle

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

from tqdm import tqdm
from typing import Tuple

# pip install openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.worksheet import Worksheet

In [ ]:
prev_codi_df = pd.read_excel('/opt/ml/input/data/raw_codishop/view/codi/codi.xlsx', engine='openpyxl')
prev_tag_df = pd.read_excel('/opt/ml/input/data/raw_codishop/view/codi/codi_tag.xlsx', engine='openpyxl')

In [ ]:
new_codi_url = pd.read_excel('/opt/ml/input/data/raw_codishop/view/item/item_rel_codi_url.xlsx', engine='openpyxl')

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(argument='--headless')
chrome_options.add_argument(argument='--no-sandbox')
chrome_options.add_argument(argument='--disable-dev-shm-usage')

driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)
driver.implicitly_wait(1.5)

In [ ]:
new_codi_url

In [ ]:
for codi_url in tqdm(new_codi_url['rel_codi_url']):
    codi_id = codi_url.split("/")[-1]
    driver.get(codi_url)

    codi_style = driver.find_element(by=By.CSS_SELECTOR, value=".style-title-box > .title").text
    
    is_codimap = driver.find_element(by=By.CSS_SELECTOR, value='.title_box > .title-page').text
    if is_codimap == "Codimap":
        codi_img_url = driver.find_elements(by=By.CSS_SELECTOR, value=".codimap-cont > .photo")
        codi_img_url = codi_img_url[-1].get_attribute('src')
    else:
        codi_img_url = driver.find_elements(by=By.CSS_SELECTOR, value=".detail_img")
        codi_img_url = codi_img_url[-1].get_attribute('src')
    # print (codi_img_url)
    
    codi_popularity = driver.find_element(by=By.CSS_SELECTOR, value=".styling_date > #view_num").text
    codi_popularity = codi_popularity.split(" ")[-1]

    codi_tags = driver.find_elements(by=By.CSS_SELECTOR, value=".styling_tag > .ui-tag-list > a")
    codi_tag_list = list()
    for codi_tag_anchor in codi_tags:
        codi_tag_list.append(codi_tag_anchor.text)

    for tag in codi_tag_list:
        codi_tag_len = len(prev_tag_df)
        prev_tag_df.loc[codi_tag_len] = [codi_id, tag]


    # print ("codi_id", codi_id)
    # print ("codi style", codi_style)
    # print ("codi img url", codi_img_url)
    # print ("codi popularity", codi_popularity)
    # print ("codi tags", codi_tag_list)

    codi_df_len = len(prev_codi_df)
    prev_codi_df.loc[codi_df_len] = [codi_id, codi_style, codi_img_url, codi_url, codi_popularity]

    # break

In [ ]:
prev_codi_df['popularity'] = prev_codi_df['popularity'].apply(lambda x : int(str(x).replace(",", "")))

In [ ]:
prev_codi_df.astype({
    'id': 'int',
    'popularity': 'int'
})

prev_tag_df.astype({
    'id': 'int',
})

In [ ]:
prev_codi_df = prev_codi_df.drop_duplicates(keep='last')
prev_tag_df = prev_tag_df.drop_duplicates(keep='last')

In [ ]:
prev_codi_df.to_excel("/opt/ml/input/data/raw_codishop/view/codi/codi.xlsx", index=False)
prev_tag_df.to_excel("/opt/ml/input/data/raw_codishop/view/codi/codi_tag.xlsx", index=False)

In [ ]:
driver.close()